In [ ]:
pip install torch

In [ ]:
pip install --user git+https://github.com/devitocodes/devito.git

In [1]:
import os

In [2]:
os.environ['DEVITO_LANGUAGE'] = 'openmp'

os.environ['OMP_SCHEDULE'] = 'STATIC'

os.environ['OMP_PROC_BIND'] = 'true'

os.environ['OMP_PLACES'] = 'cores'

In [3]:
# import libraries and tools
import torch
import numpy as np
import h5py
from time import perf_counter
from timeit import default_timer
import scipy.io
from devito import Grid, TimeFunction, Operator, Constant, Eq, solve, first_derivative, left, configuration
from examples.seismic import Model
from examples.seismic import TimeAxis
from examples.seismic import RickerSource
from examples.seismic import Receiver

In [4]:
np.random.seed(42)

# Each cross sectional area is size 3.5km x 5km
extent_x = 5.e3  
extent_z = 35.e2  

# Each cross sectional area has 207 x 201 nodes
nx = 201  
nz = 207 

# Damping layer is 20m
absorbingLayer = 20 

dx = extent_x / (nx-1) # metres
dz = extent_z / (nz-1) # metres

shape = (64, 64)
spacing = (dx, dz)
origin = (0., 0.)

t0 = 0.  # Milliseconds
T = 1000 # Milliseconds

# Source peak frequency
f0 = 0.018 # 18Hz (0.018 kHz)

# create velocity model from 3D overthrust model
hf = h5py.File('overthrust_3D_true_model.h5', 'r')
m = hf['m']
m2 = np.array(m)
m2 = np.transpose(m2, (1,2,0)) 


# create the cross sectional area velocity models
v_total = np.zeros(shape=(3204,nx, nz))
count = 0
for i in range(801):
    for j in range(4):
        v0 = m2[j*200:((j+1)*200)+1,i,:]
        # Revert squared slowness to obtain original velocity  
        v0 = 1/np.sqrt(v0)
        v_total[count] = v0           
        count+=1
print(count)
print(v_total.shape)


3204
(3204, 201, 207)


In [5]:
from plot_wave import plot_velocity
# Randomly select 1000 velocity models for training and 100 velocity models for testing
np.random.shuffle(v_total)
test_v = v_total[0:100,74:138,74:138]
train_v = v_total[999:1499,74:138,74:138]
print(test_v.shape)
print(train_v.shape)

(100, 64, 64)
(500, 64, 64)


In [6]:
# Silence the runtime performance logging
configuration['log-level'] = 'ERROR'

# Kernel dies when trying to produce 1000 samples, creating 100 instead to use as dummy data for creating FNO
for velocity_models in [test_v]: #, train_v]:
    
    N = velocity_models.shape[0]
    # shot records are: ( # of records, # of time steps in 2 seconds (nt = T/dt), grid)
    shot_records = np.zeros(shape=(N ,530, 64, 64))
    count2 = 0
    t1 = default_timer()
    for v in velocity_models:
        # Initiate Velocity Model
        model = Model(vp=v, origin=origin, shape=shape, spacing=spacing,
                      space_order=2, nbl=absorbingLayer, bcs="damp")
        
        dt = model.critical_dt
        nt = int(((T ) / dt))
        time_range = TimeAxis(start=t0, stop=T, step =dt)
        
        # Single source at the top
        src = RickerSource(name='src', grid=model.grid, f0=f0,
                                   npoint=1, time_range=time_range)

        # Source co-ordinates
        src.coordinates.data[0, 0] = 800 # Metres
        src.coordinates.data[0, 1] = 20 # Metres
        
        # Note: Receivers are not needed for FNO, learns entire wavefield

        # Wave equation for propagation
        u = TimeFunction(name="u", grid=model.grid, time_order=2, space_order=2, save=nt + 2)
        pde = model.m * u.dt2 - u.laplace + model.damp * u.dt
        stencil = Eq(u.forward, solve(pde, u.forward))

        src_term = src.inject(field=u.forward, expr=src*dt**2 / model.m)
        op = Operator([stencil] + src_term) #+ rec_term) 
        # generate solution
        op(time=nt, dt=dt)
        
        # 19:83 so that damping mask is not included in solution 
        shot_records[count2] = u.data[:,19:83,19:83]
        
        count2+= 1
        print(count2)
    
    t2 = default_timer()
    if N == 500:
            scipy.io.savemat('w_train.mat', mdict={'vel': train_v, 'sr': shot_records})
    else:
            scipy.io.savemat('w_test.mat', mdict={'vel': test_v, 'sr': shot_records})



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
